# Understand and use peformance log files in TRUST

## TRUST counters

In order to obtain statistics on the performance of a test case, **TRUST** uses counters. Counters are C++ objects. 

Their main purpose is to serve as **time watches** for parts of the code you deem important. They also track additional metrics:

**Timing statistics:**
- Minimum, maximum, average and standard deviation of the measured time per time step

**Usage metrics:**
- An integer called `count` tracking the number of times the counter has been started and stopped
- A custom integer called `quantity`, mainly used to measure the quantity of bytes exchanged during communication operations, or to store the number of iterations of the linear solver

**Identification:**
- `std::string` attributes `description` and `family`. A description is mandatory for each counter. The family attribute is by default set to `"None"` but can be used as a key for regrouping the counters you want to parse together after your computation.

**Level management:**
Counters also have a certain level. The level of a counter is represented by an integer. It makes sure that you know when you open your counter. Indeed, if a counter of level 1 is running, you can only start a counter of level 2. Otherwise, the code will stop. In the same logic, you can only close the most recently opened counter. Because of this interlock structure, counters also have another interesting metric called **alone_time**. It is the elapsed time where the considered counter is the last opened one (i.e., excluding time spent in nested counters). This metric is printed in the CSV output file and helps identify the intrinsic cost of each code section.

**Example of the level hierarchy:**
```cpp
statistics().begin_count(STD_COUNTERS::total_execution_time, -1);           // Level -1 - reserved to the total execution time counter
    statistics().begin_count(STD_COUNTERS::timeloop, 0);                    // Level 0  - reserved to counters of the global steps
        statistics().begin_count(STD_COUNTERS::convection, 1);              // Level 1  - only possible while Level 0 is running
            statistics().begin_count(STD_COUNTERS::mpi_recv, 2);            // Level 2  - only possible while Level 1 is running
            statistics().end_count(STD_COUNTERS::mpi_recv, 1 , nb_exchanged_bytes);
        statistics().end_count(STD_COUNTERS::convection);
    statistics().end_count(STD_COUNTERS::timeloop);
statistics().end_count(STD_COUNTERS::total_execution_time);
```
This ensures proper nesting and helps track where time is spent in your code hierarchy.

Those counter objects are managed by the Perf_counters class. In practice, you will only need to interact with the Perf_counters class (not with individual counter objects directly). The Perf_counters class follows a singleton pattern and a Pimpl pattern, such that the implementation of the class is hidden in the Perf_counters.cpp file. The unique instance of Perf_counters can be called inside the code by using:

```cpp
statistics()
```

The unique instance of Perf_counters will be created at the first `statistics()` call.  

The counters managed by the Perf_counters instance are separated in two types:
- the standard counters used by default in **TRUST** and identified by a `STD_COUNTERS` key (`STD_COUNTERS` is an enum class),
- custom counters that can be created inside the code and that are identified by a `std::string`. 

The basic API for counters in **TRUST** is as follows:

```cpp
#include <Perf_counters.h>

statistics().begin_count(MY_COUNTER_KEY, level);
{
    // The block of code I want to have statistics about
    std::cout << "Time elapsed since the counter has last been opened:" << statistics().get_time_since_last_open(MY_COUNTER_KEY)<< std::endl;
}
statistics().end_count(MY_COUNTER_KEY, count, quantity);
```

MY_COUNTER_KEY is either a `STD_COUNTERS` if you want to open a standard **TRUST** counter or the `std::string` that corresponds to the description of the custom counter you try to open. In the `statistics().begin_count()` function, the `level` parameter is optional. If omitted, the counter will automatically use the one defined at the creation of the counter. If you are having trouble determining the level of your counter, you can use the function `statistics().get_last_opened_counter_level()` to know the level of the last opened counter. The `count` parameter specifies how much to increment the counter's total count (i.e., how many times to record this begin/end cycle). It is set by default to 1. The `quantity` parameter specifies how much to increment the quantity attribute of your counter and is by default set to 0.


During a **TRUST** computation, **TRUST** statistics are measured through 3 main steps: 
- Computation start-up 
- Time loop
- Post-resolution

At the end of each step, the counters are reset and statistics are printed in the two files:
- `MY_TRUST_CASE_NAME.TU` 
- `MY_TRUST_CASE_NAME_csv.TU`

The first file contains aggregated stats that are the most commonly used alongside some information regarding the environment of your computation (date, OS, CPU model, GPU model if you run a GPU computation, number of CPU processors used, ...). It has been designed to be human readable, but it is not easy to parse it informatically.
The second file has been created for easily parsing each and every counter's data with your favorite csv parsing tool, for example [pandas](https://pandas.pydata.org/).


## Standard TRUST counters
Here is the list of the standard **TRUST** counters:

| Key | Description | Family | Is_communication | Is_gpu |
|-----|-------------|--------|------------------|--------|
| total_execution_time | Total time | None | False | False |
| computation_start_up | Computation start-up | None | False | False |
| timeloop | Time loop | None | False | False |
| backup_file | Back-up operations | None | False | False |
| system_solver | Linear solver resolutions Ax=B | None | False | False |
| petsc_solver | Petsc solver | None | False | False |
| implicit_diffusion | Solver for implicit diffusion | None | False | False |
| compute_dt | Computation of the time step dt | None | False | False |
| turbulent_viscosity | Turbulence model::update | None | False | False |
| convection | Convection operator | None | False | False |
| diffusion | Diffusion operator | None | False | False |
| gradient | Gradient operator | None | False | False |
| divergence | Divergence operator | None | False | False |
| rhs | Source terms | None | False | False |
| postreatment | Post-treatment operations | None | False | False |
| restart | Read file for restart | None | False | False |
| matrix_assembly | Nb matrix assembly for implicit scheme: | None | False | False |
| update_variables | Update ::mettre_a_jour | None | False | False |
| mpi_sendrecv | MPI_send_recv | MPI_sendrecv | true | False |
| mpi_send | MPI_send | MPI_sendrecv | true | False |
| mpi_recv | MPI_recv | MPI_sendrecv | true | False |
| mpi_bcast | MPI_broadcast | MPI_sendrecv | true | False |
| mpi_alltoall | MPI_alltoall | MPI_sendrecv | true | False |
| mpi_allgather | MPI_allgather | MPI_sendrecv | true | False |
| mpi_gather | MPI_gather | MPI_sendrecv | true | False |
| mpi_partialsum | MPI_partialsum | MPI_allreduce | true | False |
| mpi_sumdouble | MPI_sumdouble | MPI_allreduce | true | False |
| mpi_mindouble | MPI_mindouble | MPI_allreduce | true | False |
| mpi_maxdouble | MPI_maxdouble | MPI_allreduce | true | False |
| mpi_sumfloat | MPI_sumfloat | MPI_allreduce | true | False |
| mpi_minfloat | MPI_minfloat | MPI_allreduce | true | False |
| mpi_maxfloat | MPI_maxfloat | MPI_allreduce | true | False |
| mpi_sumint | MPI_sumint | MPI_allreduce | true | False |
| mpi_minint | MPI_minint | MPI_allreduce | true | False |
| mpi_maxint | MPI_maxint | MPI_allreduce | true | False |
| mpi_barrier | MPI_barrier | MPI_allreduce | true | False |
| gpu_library | GPU_library | GPU_library | false | true |
| gpu_kernel | GPU_kernel | GPU_kernel | false | true |
| gpu_copytodevice | GPU_copyToDevice | GPU_copy | false | true |
| gpu_copyfromdevice | GPU_copyFromDevice | GPU_copy | false | true |
| gpu_malloc_free | GPU_allocations | GPU_alloc | false | true |
| interprete_scatter | Scatter_interprete | None | true | false |
| virtual_swap | DoubleVect/IntVect::virtual_swap | None | true | False |
| read_scatter | Scatter::read_domaine | None | true | False |
| parallel_meshing | Parallel meshing | None | False | False |
| IO_EcrireFicPartageBin | write | IO | False | False |
| IO_EcrireFicPartageMPIIO | MPI_File_write_all | IO | False | False |
   

You can use them whenever you need to.

### Custom TRUST counters

As explained above, on top of standard counters, you can also create and use custom counters. To create a new custom counter, you just need to add the following in your code:
```cpp
statistics().create_custom_counter(std::string counter_description, int counter_level,  std::string counter_family = "None", bool is_comm=false, bool is_gpu=false);
```
Then, you can open and close your new counter, using `std::string counter_description` as your new counter key. All of the custom counters will be printed in both TU files.


Warning: if a custom counter already has your `std::string counter_description`, the function `statistics().create_custom_counter()` will not create another counter.

### Description of the csv file
 
The performance log file CASE_NAME_csv.TU stores the statistics in a table for each counter on a specific test case. This table has the following columns: 

"Overall_simulation_step", "Processor_Number", "Counter_family", "Counter_name",  "Counter_level", "Is_comm", "%_total_time", "total time", "t_min", "t_max", "t_SD", "time_alone", "t_alone_min", "t_alone_max", "t_alone_SD",  "count", "time_per_step", "tps_min", "tps_max", "tps_SD", "Quantity", "q_min", "q_max", "q_SD"

Column delimiter is a tabulation. The first four columns create a unique key for a row.

"Overall_simulation_step" stands respectively for the three main steps of the calculation. 

Column Is_comm enable to know if the counter is a communication counter or not. If it is equal to 1, then it is a communication counter. Otherwise, it is not.

"%_total_time" gives the percentage of time used by a peculiar counter with respect to the total time used for the main step of calculation (initialisation, resolution and post processing).

By default, averaged statistics on all processors are also available under Processor_Number == -1. For accessing the detail per processor, add **stat_per_proc_perf_log 1** in the data file. Moreover, if count is equal to zero, then the counter was not called during the computation.

SD stands for standard deviation. Min and max denote the minimum and maximum value over every processor of the accounted quantity. The "time_per_step" column stores the average time elapsed in a time step on the tracked operation.


## Exemple on the simple test case Obstacle

In [ ]:
from trustutils import run

run.TRUST_parameters("1.9.7_beta")

In [ ]:
run.reset()
case1 = run.addCase(".","Obstacle.data",nbProcs=2)
case1.partition()
run.printCases()
run.runCases()

In [ ]:
run.tablePerf()

In [ ]:
import os
os.chdir(run.BUILD_DIRECTORY)

First, we need to know the number of rows of comments there are before the table starts. Those rows always start with the character #.

In [ ]:
perf_file = open('PAR_Obstacle_csv.TU',"r") 

lines=perf_file.readlines()

nb_comment_lines=0

while (lines[nb_comment_lines][0]=='#'):
    nb_comment_lines +=1

print(nb_comment_lines)  

perf_file.closed


The file contains some white spaces, in order to make it readable by a human. They have to be removed so as to have a clean MultiIndex. To do so, three functions are created : strip, make_int and make_float. 

The column names are specified in the variable col_names. Here, by default, the same name as the first row of the table has been used, but you can customize it.

Then, the file is open using pandas library and converted into a DataFrame. A quick user guide for pandas library can be found at : https://pandas.pydata.org/docs/user_guide/10min.html

For opening the file, the function read.csv is used. The first argument is the path to the .csv file. Then, by using sep='\t' we specify that the delimiter of the table is a tabulation. Header = 0 enables to change properly the name of the colums from the first row to the ones contain in the variable col_names. Argument index_col specifies the number of columns used as a key for a row (this will create a MultiIndex). Converters are used to discard useless white spaces in the table.  


In [ ]:
import pandas as pd
import numpy as np
import math as math
import re as re

def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

def make_int(text):
    return int(text.strip('" '))

def make_float(text):
    return float(text.strip('" '))

col_names=["Overall_simulation_step", "Processor_Number", "Counter_family", "Counter_name",  "Counter_level", "Is_comm", "%_total_time", "total_time", "t_min", "t_max", "t_SD", "time alone", "t_alone_min", "t_alone_max", "t_alone_SD", "count", "time_per_step", "tps_min", "tps_max", "tps_SD", "Quantity", "q_min", "q_max", "q_SD"]

M=pd.read_csv('PAR_Obstacle_csv.TU', keep_default_na=False , sep='\t', skiprows = nb_comment_lines, header=0, names=col_names, index_col=(0,1,2,3), converters= {"Overall_simulation_step" : strip, "Processor_Number" : make_int, "Counter_family" : strip, "Counter_name" : strip, "Counter_level" : make_int, "Is_comm" : make_int, "%_total_time" : make_float, "total time" : make_float, "t_min" : make_float, "t_max" : make_float, "t_SD" : make_float, "count" : make_float, "c_min" : make_float, "c_max" : make_float, "c_SD" : make_float, "time_per_step" : make_float, "tps_min" : make_float, "tps_max" : make_float, "tps_SD" : make_float, "Quantity" : make_float, "q_min" : make_float, "q_max" : make_float, "q_SD" : make_float  })


If you have a doubt in the MultiIndex, you can print it by using :

In [ ]:
idx = M.index

print(M.index)


You can also have generic infromation about the DataFrame by using :

In [ ]:
M.info()

An advantage of the MultiIndex is that it is quite easy to make selection on partial index. First, let's create three sub DataFrames, one for each overall simulation step :

In [ ]:
M_init = M.loc[("Computation start-up statistics")]
M_reso = M.loc[("Time loop statistics")]
M_post = M.loc[("Post-resolution statistics")]

## How to reconstruct the agregated data of the .TU file

In the .Tu file, some usefull aggregated values are already computed. In the sequel, we show how to reconstruct them easily from the .csv file. First, let's print the file CASE_NAME.TU :

In [ ]:
with open('PAR_Obstacle.TU', 'r') as f:
    for lines in f:
        print(lines)

First, we need to know the number of proc used for the simulation. If statistics detailed by processor are printed, you can obtain it with:

In [ ]:
Nb_proc = np.max( idx.get_level_values(1) ) + 1    
print('Nb proc =', Nb_proc)

Else, the information is also given in the second commented line of the file:

In [ ]:
Nb_proc=0
with open('PAR_Obstacle_csv.TU', 'r') as f:
    for lines in f:
        if lines.startswith('# Number of processor used ='):
            Nb_proc = int(re.search(r'= (\d+)', lines).group(1))
            break
print('Nb proc =', Nb_proc)


In the sequel, we give an example for treating statistics of the resolution step. Just change the name of the matrix to change of overall step.

First, let's acces at the total time of the associated overall step:

In [ ]:
total_time = np.asarray(M_reso.loc[[(-1,"None","Total time")],["total_time"]])[0][0]
print("Total time of the resolution (time loop) =",total_time, "s")

The number of time steps is simply the count of the counter "Time loop". One can acces it by using:

In [ ]:
timesteps=np.asarray(M_reso.loc[[(-1,"None","Time loop")],["count"]])[0][0]
print('Number of time steps =', timesteps)

Under some conditions, such as a strictly positive number of time steps, most of the aggregated quantities of the CASE_NAME.TU file can be retrieved as follow:

In [ ]:
solveur_Axb = np.asarray(M_reso.loc[[(-1,"None","Linear solver resolutions Ax=B")],["Quantity"]])[0][0]/timesteps
print('Average number of solveur iterations per time step',solveur_Axb)

In [ ]:
solveur_diffusion_implicite = np.asarray(M_reso.loc[[(-1,"None","Solver for implicit diffusion")],["tps_max"]])[0][0]
print('Maximum time taken for solving the diffusion part per time step = ',solveur_diffusion_implicite ,"s")


In [ ]:
assemblage_matrice_implicite = np.asarray(M_reso.loc[[(-1,"None","Matrix assembly for implicit scheme:")],["tps_max"]])[0][0]
print('Maximum time taken for creating the matrix per time step, when using implicit solver = ',assemblage_matrice_implicite,"s")


In [ ]:
mettre_a_jour = np.asarray(M_reso.loc[[(-1,"None","Update ::mettre_a_jour")],["tps_max"]])[0][0]
print('Maximum time taken for updating the matrix per time step = ', mettre_a_jour, "s")


In [ ]:
operateurs_convection = np.asarray(M_reso.loc[[(-1,"None","Convection operator")],["tps_max"]])[0][0]
print('Maximum time per time step for computing the operator of convection =',operateurs_convection,'s')


In [ ]:
operateurs_diffusion =  np.asarray(M_reso.loc[[(-1,"None","Diffusion operator")],["tps_max"]])[0][0]
print('Maximum time per time step for computing the operator of convection =',operateurs_diffusion,'s')


In [ ]:
operateurs_gradient = np.asarray(M_reso.loc[[(-1,"None","Gradient operator")],["tps_max"]])[0][0]
print('Maximum time per time step for computing the gradient =',operateurs_gradient,'s')    


In [ ]:
operateurs_divergence =  np.asarray(M_reso.loc[[(-1,"None","Divergence operator")],["tps_max"]])[0][0]
print('Maximum time per time step for computing the operator divergence =',operateurs_divergence,'s')         


In [ ]:
operateurs_source = np.asarray(M_reso.loc[[(-1,"None","Source terms")],["tps_max"]])[0][0]
print('Maximum time per time step for computing source terms =',operateurs_source,'s')          


In [ ]:
operations_post_traitement =  np.asarray(M_reso.loc[[(-1,"None","Post-treatment operations")],["tps_max"]])[0][0]
print('Maximum time per time step for post treatment operations =',operations_post_traitement,'s')          


In [ ]:
calcul_dt = np.asarray(M_reso.loc[[(-1,"None","Computation of the time step dt")],["tps_max"]])[0][0]
print('Maximum time per time step for computing the time step =',calcul_dt,'s')          


In [ ]:
modele_turbulence = np.asarray(M_reso.loc[[(-1,"None","Turbulence model::update")],["tps_max"]])[0][0]
print('Maximum time per time step for trating the turbulence =',modele_turbulence,'s')          


In [ ]:
operations_sauvegarde = np.asarray(M_reso.loc[[(-1,"None","Back-up operations")],["tps_max"]])[0][0]
print('Maximum time per time step for printing back-up =',operations_sauvegarde,'s')          


In [ ]:
calcul_divers = np.asarray(M_reso.loc[[(-1,"None","Time loop")],["time alone"]])[0][0]/timesteps
calcul_divers += np.asarray(M_reso.loc[[(-1,"None","Total time")],["time alone"]])[0][0]/timesteps
print('Averaged time per time step of untracked time =',calcul_divers,'s')


In [ ]:
Nb_echange_espace_virtuel_per_ts = np.asarray(M_reso.loc[[(-1,"None","DoubleVect/IntVect::virtual_swap")],["count"]])[0][0]/timesteps
print("Average number of virtual swap (echange_espace_virtuel) per time step: ",Nb_echange_espace_virtuel_per_ts)


In [ ]:
Nb_MPI_allreduce_per_ts = (M_reso.xs('MPI_allreduce', level='Counter_family')['count'].sum())/timesteps 
print('Average number of MPI_allreduce per time step',Nb_MPI_allreduce_per_ts)


In [ ]:
Nb_solveur_per_ts = np.asarray(M_reso.loc[[(-1,"None","Linear solver resolutions Ax=B")],["count"]])[0][0]/timesteps
print('Average number of solver called per time step =',Nb_solveur_per_ts)


In [ ]:
if (np.asarray(M_reso.loc[[(-1,"None","Linear solver resolutions Ax=B")],["count"]])[0][0] > 0 ):
    Secondes_per_solveur = np.asarray(M_reso.loc[[(-1,"None","Linear solver resolutions Ax=B")],["tps_max"]])[0][0] 
    Iterations_per_solveur = np.asarray(M_reso.loc[[(-1,"None","Linear solver resolutions Ax=B")],["q_max"]])[0][0]/np.asarray(M_reso.loc[[(-1,"None","Linear solver resolutions Ax=B")],["count"]])[0][0]
    print('Maximum time per step used for solving the inversion problem =',Secondes_per_solveur)
    print('Maximum iteration number for solving the inversion problem = ',Iterations_per_solveur)

In [ ]:
if (np.asarray(M_reso.loc[[(-1,"None","Back-up operations")],["count"]])[0][0]>0):
    Nb_sauvegardes = np.asarray(M_reso.loc[[(-1,"None","Back-up operations")],["count"]])
    Data_per_sauvegarde = (Nb_proc*np.asarray(M_reso.loc[[(-1,"None","Back-up operations")],["Quantity"]]))[0][0]/(np.asarray(M_reso.loc[[(-1,"None","Back-up operations")],["count"]])[0][0] * 1024 *1024)
    print('Maximum number of back-up created :', Nb_sauvegardes)
    print('Average bytes stored per back-up :', Data_per_sauvegarde)

In [ ]:
if (np.asarray(M_reso.loc[[(-1,"GPU_copy","GPU_copyToDevice")],["count"]])>0):
    GPU_libraries = np.asarray(M_reso.loc[[(-1,"GPU_library","GPU_library")],["count"]])[0][0]/np.asarray(M_reso.loc[[(-1,"GPU_library","GPU_library")],["t_max"]])[0][0]/1024/1024/1024
    GPU_kernel = np.asarray(M_reso.loc[[(-1,"GPU_kernel","GPU_kernel")],["count"]])[0][0]/np.asarray(M_reso.loc[[(-1,"GPU_kernel","GPU_kernel")],["t_max"]])[0][0]/1024/1024/1024
    Copy_H2D = np.asarray(M_reso.loc[[(-1,"GPU_copy","GPU_copyToDevice")],["count"]])[0][0]
    print(GPU_libraries)
    print(GPU_kernel)
    print(Copy_H2D)


In [ ]:
if (M_reso.loc[(-1, "IO", "write"), "total_time"] >0):
    Debit_write_seq = Nb_proc*M_reso.loc[(-1, "IO", "write"), "Quantity"]/M_reso.loc[(-1, "IO", "write"), "total_time"]/1024/1024
    print('Sequential writing output :',Debit_write_seq, 'Mo/s')


In [ ]:
if (M_reso.loc[(-1,"IO","MPI_File_write_all"),"total_time"] >0):
    Debit_write_par = Nb_proc*M_reso.loc[(-1,"IO","MPI_File_write_all"),"Quantity"]/M_reso.loc[(-1,"IO","MPI_File_write_all"),"total time"]/1024/1024
    print('Parallel writing output :',Debit_write_par, 'Mo/s')


In [ ]:
if (M_reso[M_reso['Is_comm'] == 1]['count'].sum()> 0):
    Communications_avg = M_reso[M_reso['Is_comm'] == 1]['total_time'].sum()/ M_reso.loc[(-1,"None","Total time"),"total_time"]
    Communications_max = M_reso[M_reso['Is_comm'] == 1]['t_max'].sum()/ M_reso.loc[(-1,"None","Total time"),"t_min"]
    Communications_min = M_reso[M_reso['Is_comm'] == 1]['t_min'].sum()/ M_reso.loc[(-1,"None","Total time"),"t_max"]
    print('Average percent of time of communications = ',Communications_avg)
    print('Minimum percent of time of communications = ',Communications_min)
    print('Maximum percent of time of communications = ',Communications_max)
    if (M_reso.loc[(-1,"MPI_sendrecv", "MPI_send_recv"),"total_time"] > 0):
        max_bandwidth = 1.0e-6 * (M_reso.loc[(-1,"MPI_sendrecv", "MPI_send_recv"),"Quantity"])/M_reso.loc[(-1,"MPI_sendrecv", "MPI_send_recv"),"t_min"]
        print('Evaluation of the maximum badnwidth = ',max_bandwidth, ' MB/s')
    Nb_MPI_senrecv_per_ts = (M_reso.xs('MPI_sendrecv', level='Counter_family')['count'].sum())/timesteps 
    bytes_MPI_senrecv_per_ts = (M_reso.xs('MPI_sendrecv', level='Counter_family')['Quantity'].sum())/timesteps 
    Total_network_traffic = Nb_proc * 1.0e-6 * bytes_MPI_senrecv_per_ts / timesteps
    if(Nb_MPI_senrecv_per_ts>0):
        Average_message_size = bytes_MPI_senrecv_per_ts / Nb_MPI_senrecv_per_ts
    print('Average total size of send messages per time step = ',Total_network_traffic, ' B per time step')
    print('Average size of send messages per exchange = ',Average_message_size, ' MB/s')